# AI Orchestrator - RAG Advance

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Prep

In [2]:
import os
from openai import AzureOpenAI

#MODEL = "gpt-4o"
MODEL = "gpt-35-turbo-1106"
client = AzureOpenAI(
    api_key=os.environ['AZURE_OPENAI_KEY2'],
    #api_version="2024-10-01-preview",
    api_version="2023-12-01-preview",
    azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT2']
)

In [3]:
def _chat_gpt(messages, model=MODEL, temp=0.0, topp=0.1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=4000,
        top_p=topp
    )   
    
    return response.choices[0].message.content

In [4]:
from azure.core.credentials import AzureKeyCredential
import os

endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

index_name = os.environ["AZSCH_INDEX_NAME"]

In [5]:
# prompt templates
from jinja2 import Template
from common import parse_chat

with open('./determine_reply.jinja2') as file:
    reply_template = file.read()

with open('./determine_intent.jinja2') as file:
    intent_template = file.read()

## Intent

In [6]:
def determine_intent(query, intent_history):
    prompt = Template(intent_template, trim_blocks=True, keep_trailing_newline=True).render(
        intent_history=intent_history,
        query=query
    )

    messages = parse_chat(prompt)
    
    return _chat_gpt(messages)

def extract_intent(input: str, query: str) -> str:
  entries = None
  if 'Single Intents:' in input:
    entries = input.split('Single Intents:', 2)
  elif 'Single Intent:' in input:
    entries = input.split('Single Intent:', 2)
  
  if entries and len(entries) == 2:
    return {
      "current_message_intent": entries[0].strip(),
      "search_intents": entries[1].strip()
    }
  return {
      "current_message_intent": query,
      "search_intents": query
  }

In [7]:
import json

def get_search_intent(query, intent_history):
    raw_intent = determine_intent(query, intent_history)
    intent = extract_intent(raw_intent, query)
    js = json.loads(intent['search_intents'])
    
    return js[0] if len(js) != 0 else "", raw_intent

In [8]:
intent_history = []
query = "What is Assistant API"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)
intent

'what is Assistant API?'

In [9]:
query = "Show some sample code in python"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)
print(intent, "\n", raw_intent)

show some sample code in python 
 Show some sample code in python
Single Intents: ["show some sample code in python"]


In [10]:
intent_history

['what is Assistant API?', 'show some sample code in python']

### Search

In [11]:
from aisearch import AISearch
from common import format_retrieved_documents

def get_context(intent):
    search = AISearch(endpoint, index_name, credential)
    docs = search.get_results(intent)
    documentation, metadata = format_retrieved_documents(docs, 4000)

    return documentation, metadata

In [12]:
intent = "show sample code for Assistant API in python"

In [13]:
documentation, metadata = get_context(intent)
print(documentation)

{'retrieved_documents': [{'[doc0]': {'title': 'Quickstart: Use the OpenAI Service via the Python SDK', 'content': ':::image type="content" source="../media/quickstarts/endpoint.png" alt-text="Screenshot of the overview blade for an OpenAI Resource in the Azure portal with the endpoint & access keys location circled in red." lightbox="../media/quickstarts/endpoint.png":::\n\nCreate and assign persistent environment variables for your key and endpoint.\n\n[!INCLUDE [environment-variables](environment-variables.md)]\n\n## Create an assistant\n\nIn our code we are going to specify the following values:\n\n| **Name** | **Description** |\n|:---|:---|\n| **Assistant name** | Your deployment name that is associated with a specific model. |\n| **Instructions** | Instructions are similar to system messages this is where you give the model guidance about how it should behave and any context it should reference when generating a response. You can describe the assistant\'s personality, tell it what

In [14]:
metadata

[{'title': 'Quickstart: Use the OpenAI Service via the Python SDK',
  'link': '<sup>[1](/static/assistants-python.md)</sup>'},
 {'title': 'Quickstart: Use the OpenAI Service via the Python SDK',
  'link': '<sup>[2](/static/assistants-python.md)</sup>'},
 {'title': 'How to create Assistants with Azure OpenAI Service',
  'link': '<sup>[3](/static/assistant.md)</sup>'}]

### generate answer

In [15]:
def generate_response(chat_history, intent, documentation):
    prompt = Template(reply_template, trim_blocks=True, keep_trailing_newline=True).render(
        chat_history=chat_history,
        documentation=documentation,
        user_query=intent
    )
    
    messages = parse_chat(prompt)
    response = _chat_gpt(messages)

    return response

In [16]:
chat_history = []
output = generate_response(chat_history, intent, documentation)

In [18]:
from IPython.display import display, Markdown
display(Markdown(output))

The following Python code demonstrates how to create an assistant using the Azure OpenAI service:

```python
import os
import time
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.
)

# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
```
This code creates an assistant, initiates a conversation thread, adds a user question to the thread, runs the thread, and retrieves the status and messages from the assistant[doc0][doc2].

In [21]:
query = "assistant api sample code in rest api"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)

print(intent)

In [22]:
intent = "assistant api sample code in rest api"
documentation = get_context(intent)

output = generate_response(chat_history, intent, documentation)

In [23]:
print(output)

The sample code for creating an assistant using the Azure OpenAI Assistants REST API is as follows:

```console
curl https://YOUR_RESOURCE_NAME.openai.azure.com/openai/assistants?api-version=2024-02-15-preview \
  -H "api-key: $AZURE_OPENAI_KEY" \
  -H "Content-Type: application/json" \
  -d '{
    "instructions": "You are an AI assistant that can write code to help answer math questions.",
    "name": "Math Assist",
    "tools": [{"type": "code_interpreter"}],
    "model": "gpt-4-1106-preview"
  }'
```
This code demonstrates how to create an assistant by providing instructions, a name, tools (such as code interpreter), and the model to be used [doc1][2].

If you need further information, you can refer to the [Azure OpenAI Assistants API samples](https://github.com/Azure-Samples/azureai-samples/tree/main/scenarios/Assistants) [doc0][3].
